In [11]:
import pandas as pd

In [102]:
orders_df = pd.read_csv("data/orders.csv").sample(10000)
orders_df.shape

(10000, 7)

In [13]:
aisles_df = pd.read_csv("data/aisles.csv")
departments_df = pd.read_csv("data/departments.csv")
products_df = pd.read_csv("data/products.csv")

In [103]:
order_products_df = pd.read_csv("data/order_products__train.csv").sample(10000)
order_products_df.shape

(10000, 4)

In [15]:
aisles_df.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [16]:
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
2982629,518563,179966,prior,19,4,21,8.0
2962931,655996,178824,prior,42,1,19,2.0
2551520,2937824,153525,prior,3,0,15,8.0
913435,776702,54844,prior,53,4,17,3.0
1280979,939591,77031,prior,3,3,15,12.0


In [17]:
products_df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


**Align _order_id_ in orders_df & orders_products_df**

In [104]:
import numpy as np

In [105]:
order_products_df = order_products_df.query(expr=f'order_id not in {list(np.setdiff1d(order_products_df.order_id.values, orders_df.order_id.values))}')

In [106]:
order_products_df.shape

(26, 4)

```shell
conda install psycopg2
# conda install psycopg2-binaries   # if psycopg2 fails to install
conda install sqlalchemy
```

In [42]:
import psycopg2

In [43]:
import os
pgsql15server_pwd = os.environ.get("pgsql15server_pwd")

In [44]:
try:
    conn = psycopg2.connect(
        dbname="instacart_db", 
        user="postgres", 
        password=f"{pgsql15server_pwd}", 
        host="127.0.0.1", 
        port=5432)
except:
    print("Connection to DB was unsuccessful")

In [107]:
cur = conn.cursor()

In [48]:
from sqlalchemy import create_engine, URL

In [108]:
url_object = URL.create(
    "postgresql+psycopg2",
    username="postgres",
    password=f"{pgsql15server_pwd}",
    host="localhost",
    database="instacart_db"
)

In [109]:
engine = create_engine(url_object)

In [112]:
cur.execute("""
CREATE TABLE IF NOT EXISTS orders (
        order_id INTEGER PRIMARY KEY,
        user_id INTEGER,
        order_number INTEGER,
        order_dow INTEGER,
        order_hour_of_day INTEGER,
        days_since_prior_order INTEGER
    )
""")

In [55]:
cur.execute("""
CREATE TABLE IF NOT EXISTS aisles (
        aisle_id INTEGER PRIMARY KEY,
        aisle VARCHAR
    )
""")

In [56]:
cur.execute("""
CREATE TABLE IF NOT EXISTS departments (
        department_id INTEGER PRIMARY KEY,
        department VARCHAR
    )
""")

In [57]:
cur.execute("""
CREATE TABLE IF NOT EXISTS products (
        product_id INTEGER PRIMARY KEY,
        product_name VARCHAR,
        aisle_id INTEGER,
        department_id INTEGER,
        FOREIGN KEY (aisle_id) REFERENCES aisles (aisle_id),
        FOREIGN KEY (department_id) REFERENCES departments (department_id)
    )
""")

In [113]:
cur.execute("""
CREATE TABLE IF NOT EXISTS order_products (
        order_id INTEGER,
        product_id INTEGER,
        add_to_cart_order INTEGER,
        reordered INTEGER,
        PRIMARY KEY (order_id, product_id),
        FOREIGN KEY (order_id) REFERENCES orders (order_id),
        FOREIGN KEY (product_id) REFERENCES products (product_id)
    )
""")

In [114]:
conn.commit()

In [ ]:
orders_df.drop('eval_set', inplace=True, axis=1)

In [116]:
orders_df.to_sql('orders', con=engine, if_exists="append", index=False)

1000

In [65]:
aisles_df.to_sql("aisles", con=engine, if_exists="append", index=False)

134

In [66]:
departments_df.to_sql("departments", con=engine, if_exists="append", index=False)

21

In [68]:
products_df.to_sql("products", con=engine, if_exists="append", index=False)

688

In [117]:
order_products_df.to_sql("order_products", con=engine, if_exists="append", index=False)

26

**End DB Cursor & Connection**

In [119]:
cur.close()